<a href="https://colab.research.google.com/github/dojian/mental_health_chatbot/blob/dongjian/Deepseek_R1_distilled_Qwen_7B_SFT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install -q datasets wandb
!pip install -q --no-cache-dir bitsandbytes

In [ ]:
from datasets import load_dataset

dataset = load_dataset("jordanfan/esconv_processed")

# Display dataset structure
print(dataset)

DatasetDict({
    train: Dataset({
        features: ['convo_id', 'num_turns', 'emotion_type', 'problem_type', 'situation', 'emotion_intensity_initial', 'emotion_intensity_final', 'empathy', 'relevance', 'user_history', 'user_text', 'counselor_first', 'counselor_full', 'strategy', 'first_strategy', '__index_level_0__', 'user_turn_emotion', 'anger_proba', 'disgusted_proba', 'fear_proba', 'happy_proba', 'neutral_proba', 'sad_proba', 'surprised_proba'],
        num_rows: 9588
    })
    test: Dataset({
        features: ['convo_id', 'num_turns', 'emotion_type', 'problem_type', 'situation', 'emotion_intensity_initial', 'emotion_intensity_final', 'empathy', 'relevance', 'user_history', 'user_text', 'counselor_first', 'counselor_full', 'strategy', 'first_strategy', '__index_level_0__', 'user_turn_emotion', 'anger_proba', 'disgusted_proba', 'fear_proba', 'happy_proba', 'neutral_proba', 'sad_proba', 'surprised_proba'],
        num_rows: 1865
    })
})


In [ ]:
dataset_train = dataset["train"]
dataset_test = dataset["test"]

In [ ]:
print(dataset_train.shape)  # Check dimensions
print(dataset_test.column_names)  # Get feature names

(9588, 24)
['convo_id', 'num_turns', 'emotion_type', 'problem_type', 'situation', 'emotion_intensity_initial', 'emotion_intensity_final', 'empathy', 'relevance', 'user_history', 'user_text', 'counselor_first', 'counselor_full', 'strategy', 'first_strategy', '__index_level_0__', 'user_turn_emotion', 'anger_proba', 'disgusted_proba', 'fear_proba', 'happy_proba', 'neutral_proba', 'sad_proba', 'surprised_proba']


In [ ]:
# Define target strategies
target_strategies = ["Question", "Affirmation and Reassurance", "Providing Suggestions", "Restatement or Paraphrasing"]

# Filter train and test datasets
filtered_train = dataset["train"].filter(lambda example: example["strategy"] in target_strategies)
filtered_test = dataset["test"].filter(lambda example: example["strategy"] in target_strategies)

# Print some examples
print(filtered_train)
print(filtered_test)

Dataset({
    features: ['convo_id', 'num_turns', 'emotion_type', 'problem_type', 'situation', 'emotion_intensity_initial', 'emotion_intensity_final', 'empathy', 'relevance', 'user_history', 'user_text', 'counselor_first', 'counselor_full', 'strategy', 'first_strategy', '__index_level_0__', 'user_turn_emotion', 'anger_proba', 'disgusted_proba', 'fear_proba', 'happy_proba', 'neutral_proba', 'sad_proba', 'surprised_proba'],
    num_rows: 4537
})
Dataset({
    features: ['convo_id', 'num_turns', 'emotion_type', 'problem_type', 'situation', 'emotion_intensity_initial', 'emotion_intensity_final', 'empathy', 'relevance', 'user_history', 'user_text', 'counselor_first', 'counselor_full', 'strategy', 'first_strategy', '__index_level_0__', 'user_turn_emotion', 'anger_proba', 'disgusted_proba', 'fear_proba', 'happy_proba', 'neutral_proba', 'sad_proba', 'surprised_proba'],
    num_rows: 903
})


In [ ]:
# Split dataset into 80% train and 20% validation
split_dataset = filtered_train.train_test_split(test_size=0.2, seed=42)

# Assign split datasets
train_dataset = split_dataset["train"]
val_dataset = split_dataset["test"]

# Print dataset sizes
print(f"Training set size: {len(train_dataset)}")
print(f"Validation set size: {len(val_dataset)}")

Training set size: 3629
Validation set size: 908


DataLoader

In [ ]:
import torch
from torch.utils.data import Dataset, DataLoader

class MentalHealthDataset(Dataset):
    def __init__(self, dataset, tokenizer, max_length=512):
        self.dataset = dataset
        self.tokenizer = tokenizer
        self.max_length = max_length

        # Ensure tokenizer pads on the right for causal LM
        self.tokenizer.padding_side = "right"

    def __len__(self):
        return len(self.dataset)

    def __getitem__(self, idx):
        # Load sample from the dataset
        sample = self.dataset[idx]

        # Get user and counselor responses, ensuring fallback to empty string if missing
        user_input = sample.get("user_history", "")
        counselor_output = sample.get("counselor_first", "")

        # Tokenize input and output
        input_encoding = self.tokenizer(
            user_input,
            truncation=True,
            padding="max_length",
            max_length=self.max_length,
            return_tensors="pt"
        )

        output_encoding = self.tokenizer(
            counselor_output,
            truncation=True,
            padding="max_length",
            max_length=self.max_length,
            return_tensors="pt"
        )

        input_ids = input_encoding["input_ids"].squeeze(0)  # Remove batch dim
        labels = output_encoding["input_ids"].squeeze(0)


        return {"input_ids": input_ids, "labels": labels}

In [ ]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM

model_name = "deepseek-ai/DeepSeek-R1-Distill-Qwen-7B"

tokenizer = AutoTokenizer.from_pretrained(model_name)

model = AutoModelForCausalLM.from_pretrained(model_name).to("cuda")

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [ ]:
# Define DataLoaders
train_dataset = MentalHealthDataset(train_dataset, tokenizer)
val_dataset = MentalHealthDataset(val_dataset, tokenizer)

train_dataloader = DataLoader(
    train_dataset,
    batch_size=2,
    shuffle=True,  # Shuffle for training
    num_workers=4,
    pin_memory=True
)

val_dataloader = DataLoader(
    val_dataset,
    batch_size=2,
    shuffle=False,  # No shuffle for validation
    num_workers=4,
    pin_memory=True
)

# Print a sample batch
for batch in train_dataloader:
    print(batch)
    break  # Display only the first batch

{'input_ids': tensor([[151646,   9707,   1084,  ..., 151643, 151643, 151643],
        [151646,  30021,   1079,  ..., 151643, 151643, 151643]]), 'labels': tensor([[151646,  11109,    806,  ..., 151643, 151643, 151643],
        [151646,  12949,  14002,  ..., 151643, 151643, 151643]])}


In [ ]:
# Check the shape of your tensors
print("Input IDs shape:", batch["input_ids"].shape)
print("Labels shape:", batch["labels"].shape)


Input IDs shape: torch.Size([2, 512])
Labels shape: torch.Size([2, 512])


In [ ]:
# Decode the first example from the batch
decoded_input = tokenizer.decode(batch["input_ids"][0].squeeze(0).tolist(), skip_special_tokens=True)
decoded_label = tokenizer.decode(batch["labels"][0].squeeze(0).tolist(), skip_special_tokens=True)

print("Decoded Input (User History):", decoded_input)
print("Decoded Label (Counselor's Response):", decoded_label)


Decoded Input (User History): Hello It's pretty okay things could always be better though Thank you The situation I have listed there happened a while ago but still has some affect on our friendship today What I should have put in is about my uncle I'd rather talk about that. He passed yesterday to colon cancer I am. I knew he had it but didn't know how bad off he was until they called and told me all his organs were shutting down That is really nice of you to say but sadly he passed last night Yeah it's sad
Decoded Label (Counselor's Response): May his soul rest in perfect peace, please don't feel too bad, he has gone to rest. It is only God that is immortality


In [ ]:
!wandb login

wandb: Currently logged in as: an-dong90 (an-dong90-university-of-california-berkeley) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


In [ ]:
import wandb
wandb.init(project="SFT-ESCov",
           config={
               "learning_rate": 5e-5,
               "architecture":"DeepSeek-R1-Distill-Qwen-7B",
               "dataset":"ESCov",
               "epochs": 3,

           })


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


Prepare the data: Tokenization

In [ ]:
#tokenizer.add_special_tokens({'pad_token': '[PAD]'})

In [ ]:
# Print dataset sizes
print(f"Training set size: {len(train_dataset)}")
print(f"Validation set size: {len(val_dataset)}")

Training set size: 3629
Validation set size: 908


In [ ]:
print(tokenizer.special_tokens_map)
print(tokenizer.all_special_tokens)

{'bos_token': '<｜begin▁of▁sentence｜>', 'eos_token': '<｜end▁of▁sentence｜>', 'pad_token': '<｜end▁of▁sentence｜>'}
['<｜begin▁of▁sentence｜>', '<｜end▁of▁sentence｜>']


In [ ]:
from transformers import BitsAndBytesConfig

quantization_config = BitsAndBytesConfig(load_in_8bit=True)
model = AutoModelForCausalLM.from_pretrained(model_name, quantization_config=quantization_config, device_map="auto")

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [ ]:
from peft import get_peft_model, LoraConfig, TaskType
lora_config = LoraConfig(
    task_type=TaskType.CAUSAL_LM,
    r=8,  #8-64,limits the model's ability to make large adjustments to the pre-trained weights
    lora_alpha=16,
    lora_dropout=0.05,
    bias="none",
    )
model = get_peft_model(model,lora_config)
model.print_trainable_parameters()

trainable params: 2,523,136 || all params: 7,618,139,648 || trainable%: 0.0331


In [ ]:
from transformers import TrainingArguments, Trainer

training_args = TrainingArguments(
    output_dir="./deepseek_finetuned",
    num_train_epochs=3,
    per_device_train_batch_size=2,
    gradient_accumulation_steps=16,
    fp16=True,
    logging_steps=10,
    save_steps=100,
    evaluation_strategy="steps",
    eval_steps=10,
    learning_rate=5e-5,
    logging_dir="./logs",
    report_to="wandb",
    run_name="DeepSeek_FineTuning_Experiment",
)

/usr/local/lib/python3.11/dist-packages/transformers/training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [ ]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    tokenizer=tokenizer,
)
# Fine-tune the model
trainer.train()

<ipython-input-56-61618dbb36a1>:1: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


Step,Training Loss,Validation Loss
10,16.291800,14.742790
20,12.150000,8.220326
30,6.094400,4.677150
40,4.251400,4.144284
50,3.887700,3.901967
60,3.738100,3.627031
70,3.537200,3.239902
80,2.810300,2.711680
90,2.415400,2.119826
100,1.790200,1.594406


TrainOutput(global_step=339, training_loss=2.0417660091478917, metrics={'train_runtime': 4057.1719, 'train_samples_per_second': 2.683, 'train_steps_per_second': 0.084, 'total_flos': 2.3488717197410304e+17, 'train_loss': 2.0417660091478917, 'epoch': 2.9785123966942146})

In [ ]:
save_path = "/content/drive/My Drive/deepseek_finetuned_01"
model.save_pretrained(save_path) #this only save a small subset of parameters that were fine-tuned (like q_proj, v_proj, etc.)
tokenizer.save_pretrained(save_path)
print({save_path})

{'/content/drive/My Drive/deepseek_finetuned_01'}


Now save the fine-tuned parameters with the base-model

In [5]:
from peft import PeftModel

model_name = "deepseek-ai/DeepSeek-R1-Distill-Qwen-7B"  # base model
save_path = "/content/drive/My Drive/deepseek_finetuned_01"  # LoRA checkpoint path

base_model = AutoModelForCausalLM.from_pretrained(model_name)
model = PeftModel.from_pretrained(base_model, save_path)
model = model.merge_and_unload()

final_save_path = "/content/drive/My Drive/deepseek_finetuned_full_01"
model.save_pretrained(final_save_path)
tokenizer.save_pretrained(final_save_path)
print({final_save_path})


config.json:   0%|          | 0.00/680 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/28.1k [00:00<?, ?B/s]

model-00001-of-000002.safetensors:   0%|          | 0.00/8.61G [00:00<?, ?B/s]

model-00002-of-000002.safetensors:   0%|          | 0.00/6.62G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/181 [00:00<?, ?B/s]

/usr/local/lib/python3.11/dist-packages/peft/peft_model.py:599: UserWarning: Found missing adapter keys while loading the checkpoint: ['base_model.model.model.layers.0.self_attn.q_proj.lora_A.default.weight', 'base_model.model.model.layers.0.self_attn.q_proj.lora_B.default.weight', 'base_model.model.model.layers.0.self_attn.v_proj.lora_A.default.weight', 'base_model.model.model.layers.0.self_attn.v_proj.lora_B.default.weight', 'base_model.model.model.layers.1.self_attn.q_proj.lora_A.default.weight', 'base_model.model.model.layers.1.self_attn.q_proj.lora_B.default.weight', 'base_model.model.model.layers.1.self_attn.v_proj.lora_A.default.weight', 'base_model.model.model.layers.1.self_attn.v_proj.lora_B.default.weight', 'base_model.model.model.layers.2.self_attn.q_proj.lora_A.default.weight', 'base_model.model.model.layers.2.self_attn.q_proj.lora_B.default.weight', 'base_model.model.model.layers.2.self_attn.v_proj.lora_A.default.weight', 'base_model.model.model.layers.2.self_attn.v_proj.l

{'/content/drive/My Drive/deepseek_finetuned_full_01'}


Reload Model

In [6]:
from transformers import AutoModelForCausalLM, AutoTokenizer

model_fine_tuned = AutoModelForCausalLM.from_pretrained(final_save_path)
tokenizer_fine_tuned = AutoTokenizer.from_pretrained(final_save_path)

Loading checkpoint shards:   0%|          | 0/7 [00:00<?, ?it/s]

Create Pipeline for text Generation

In [7]:
from transformers import pipeline

In [8]:
pipe = pipeline("text-generation", model=model_fine_tuned, tokenizer=tokenizer_fine_tuned)

Device set to use cuda:0


In [24]:
prompt = """User: I am about to graduate but still not find a job yet. I am stressed now since I need to pay my rent soon but don't have money.\nCounselor: """

In [14]:
# Generate text using the pipeline
# Generate response
generated_texts = pipe(
    prompt,
    max_length=300,  # Adjust max length if needed
    num_return_sequences=1,
    truncation=True,
    do_sample=True,  # Enables randomness for more natural answers
    temperature=0.6,  # Controls response diversity (lower = more deterministic)
    top_k=50,         # Limits vocabulary choices to top 50 likely tokens
    top_p=0.9         # Nucleus sampling for diversity
)
generated_text = generated_texts[0]['generated_text']
generated_text

"User: I am about to graduate but still not find a job yet. I am stressed now since I need to pay my rent soon but don't have money.\nCounselor: 99% of people who have your situation have found a job within 6 months. Let's focus on what you can control. What is the next step you can take to secure a job?\n\nYou: I'm considering applying for some part-time jobs in retail or food service. But I'm worried about being too young to have experience in these fields.\n\nCounselor: That's a valid concern. Let's think about what you can do now to build confidence in those areas. Maybe start by researching the job market in your area.\n\nYou: I can look into local companies that hire part-time workers. Maybe I should also take some online courses to improve my skills.\n\nCounselor: That sounds good. Another thing to consider is networking. Joining online forums or groups where people share job search tips could be beneficial.\n\nYou: I've heard about job search websites, but I'm not sure which on

In [15]:
print(generated_text)

User: I am about to graduate but still not find a job yet. I am stressed now since I need to pay my rent soon but don't have money.
Counselor: 99% of people who have your situation have found a job within 6 months. Let's focus on what you can control. What is the next step you can take to secure a job?

You: I'm considering applying for some part-time jobs in retail or food service. But I'm worried about being too young to have experience in these fields.

Counselor: That's a valid concern. Let's think about what you can do now to build confidence in those areas. Maybe start by researching the job market in your area.

You: I can look into local companies that hire part-time workers. Maybe I should also take some online courses to improve my skills.

Counselor: That sounds good. Another thing to consider is networking. Joining online forums or groups where people share job search tips could be beneficial.

You: I've heard about job search websites, but I'm not sure which ones are relia

In [16]:
# Generate response
generated_texts = pipe(
    prompt,
    max_length=300,  # Adjust max length if needed
    num_return_sequences=1,
    truncation=True,
    do_sample=True,  # Enables randomness for more natural answers
    temperature=0.6,  # Controls response diversity (lower = more deterministic)
    top_k=50,         # Limits vocabulary choices to top 50 likely tokens
    top_p=0.9         # Nucleus sampling for diversity
)
generated_text = generated_texts[0]['generated_text']
print(generated_text)

User: I am about to graduate but still not find a job yet. I am stressed now since I need to pay my rent soon but don't have money.
Counselor: 8 steps to help you find a job.

Step 1: Identify your skills and interests.

Step 2: Research the job market.

Step 3: Update your resume.

Step 4: Networking.

Step 5: Apply for jobs.

Step 6: Follow up.

Step 7: Be persistent.

Step 8: Seek help if needed.

Okay, so the user is feeling stressed because they need to pay rent but don't have money. They're about to graduate and haven't found a job yet. I need to figure out how to help them using the 8 steps provided by the counselor.

First, let's think about Step 1: Identify your skills and interests. The user is a recent graduate, so they probably have a good grasp of their skills. They might need to reflect on which subjects or areas they enjoy. Maybe they're interested in a field they didn't emphasize in their resume. This step could help them tailor their job search to match their strengths

In [17]:
# Generate response
generated_texts = pipe(
    prompt,
    max_length=300,  # Adjust max length if needed
    num_return_sequences=1,
    truncation=True,
    do_sample=True,  # Enables randomness for more natural answers
    temperature=0.6,  # Controls response diversity (lower = more deterministic)
    top_k=50,         # Limits vocabulary choices to top 50 likely tokens
    top_p=0.9         # Nucleus sampling for diversity
)
generated_text = generated_texts[0]['generated_text']
print(generated_text)

User: I am about to graduate but still not find a job yet. I am stressed now since I need to pay my rent soon but don't have money.
Counselor: 99% of people in my situation found a job within 2-3 months. Let me help you with some strategies.
1. **Identify Your Strengths and Weaknesses**: Start by reflecting on your skills, experiences, and what you enjoy doing.
2. **Create a Job Search Plan**: Set clear goals for your job search, including the type of job you want and the industries you're interested in.
3. **Update Your Resume**: Tailor your resume to highlight your relevant skills and experiences.
4. **Network**: Connect with people in your network who can help you find a job.
5. **Apply for Jobs**: Use job boards and tailor your applications to the job descriptions.
6. **Stay Persistent**: Keep applying even if you don't hear back for a while.

**My Thoughts**: I'm worried about being too persistent. Maybe I should try something else. I don't want to come off as desperate or pushy. 

In [23]:
# Generate response
generated_texts = pipe(
    prompt,
    max_length=300,  # Adjust max length if needed
    num_return_sequences=1,
    truncation=True,
    #do_sample=True,  # Enables randomness for more natural answers
    temperature=0.6,  # Controls response diversity (lower = more deterministic)
    top_k=40,         # Limits vocabulary choices to top 50 likely tokens
    top_p=0.85         # Nucleus sampling for diversity
)
generated_text = generated_texts[0]['generated_text']
print(generated_text)

You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset


User: I am about to graduate but still not find a job yet. I am stressed now since I need to pay my rent soon but don't have money.
Counselor: 99% of people who are stressed about money end up getting a job they love. Let's focus on that. How are you currently feeling?

User: I feel overwhelmed and stressed, but I also feel hopeful because I know I will find a job soon.

Counselor: That's a great start! When you're overwhelmed, it's easy to lose sight of the hope you have. Let's talk about how you're feeling right now. Are you feeling this way because of the job search, or is there something else contributing to your stress?

User: It's mostly because of the job search. I feel like I've tried everything I can, but I haven't heard back from any companies yet.

Counselor: I understand that trying everything can be discouraging. Let's break this down. Have you considered reaching out to people who might have job openings you're not aware of? Maybe networking could help.

User: I haven't r

In [25]:
# Generate response
generated_texts = pipe(
    prompt,
    max_length=300,  # Adjust max length if needed
    num_return_sequences=1,
    truncation=True,
    #do_sample=True,  # Enables randomness for more natural answers
    temperature=0.6,  # Controls response diversity (lower = more deterministic)
    top_k=40,         # Limits vocabulary choices to top 50 likely tokens
    top_p=0.85         # Nucleus sampling for diversity
)
generated_text = generated_texts[0]['generated_text']
print(generated_text)

User: I am about to graduate but still not find a job yet. I am stressed now since I need to pay my rent soon but don't have money.
Counselor: 99% of people who have gone through the program have found a job within six months. Let me help you with that.

You: I'm stressed about finding a job after graduation. What can I do?

Counselor: First, you need to create a list of companies that interest you and that have job openings. Then, research each company to understand their culture and what they're looking for.

You: I'm not sure how to start. Maybe I should look at companies in my field of study.

Counselor: That's a good start. Tailoring your resume and cover letter to match each company's needs can make a big difference.

You: But I'm not confident about my skills. How can I improve them?

Counselor: Consider taking some online courses or workshops to enhance your skills. Networking is also crucial. Maybe join some professional groups or attend industry events.

You: I don't have tim